In [3]:
# === P24: LR (elastic-net, saga) con RepeatedStratifiedKFold + Platt ===
import json, pickle, re
import numpy as np, pandas as pd
from pathlib import Path

from sklearn.model_selection import RepeatedStratifiedKFold, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.impute import SimpleImputer

# ---------- Config ----------
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Monta Drive si hace falta
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

BASE_DIR = Path("/content/drive/MyDrive/CognitivaAI")
OUT = BASE_DIR / "p24_meta_simple"
OUT.mkdir(parents=True, exist_ok=True)

VAL_LABELS  = BASE_DIR/"p22_meta_ablation/p22_val_calibrations.csv"
TEST_LABELS = BASE_DIR/"p22_meta_ablation/p22_test_calibrations.csv"

# Candidatos de features paciente (se fusionan si existen)
CANDIDATE_FEATURE_FILES_VAL = [
    BASE_DIR/"p11_alt_backbones/val_patient_features_backbones.csv",
    # Añade aquí otros CSV de features VAL si los tienes
]
CANDIDATE_FEATURE_FILES_TEST = [
    BASE_DIR/"p11_alt_backbones/test_patient_features_backbones.csv",
    # Añade aquí otros CSV de features TEST si los tienes
]

# ---------- Helpers ----------
# --- Parche robusto para cargar labels de VAL/TEST (acepta múltiples nombres) ---
# --- Reemplazar load_labels por esta versión (maneja BOM y normaliza nombres) ---
def load_labels(path_csv):
    # intenta leer con utf-8-sig para eliminar BOM; si falla, fallback a utf-8
    try:
        df = pd.read_csv(path_csv, encoding='utf-8-sig')
    except Exception:
        df = pd.read_csv(path_csv)
    # limpia BOM en cada cabecera y recorta espacios
    def cleanse(s):
        return str(s).replace('\ufeff','').strip()
    df.columns = [cleanse(c) for c in df.columns]
    lower = {c.lower(): c for c in df.columns}

    # patient_id
    for key in ['patient_id','patientid','pid','patient']:
        if key in lower:
            pid_col = lower[key]; break
    else:
        raise AssertionError(f"No encuentro columna de paciente en {path_csv}. Cabeceras: {list(df.columns)}")

    # y_true
    for key in ['y_true','y','target','label','truth','gt']:
        if key in lower:
            y_col = lower[key]; break
    else:
        # última oportunidad: alguna col numérica binaria
        y_col = None
        for c in df.columns:
            s = pd.to_numeric(df[c], errors='coerce')
            uniq = s.dropna().unique()
            if len(uniq)==2:
                y_col = c; break
        assert y_col is not None, f"No encuentro etiqueta binaria en {path_csv}. Cabeceras: {list(df.columns)}"

    # cohort (si falta, inferir del patient_id)
    if 'cohort' not in df.columns:
        def infer_cohort(pid):
            s = str(pid).upper()
            return 'OAS1' if s.startswith('OAS1') else ('OAS2' if s.startswith('OAS2') else 'ALL')
        df['cohort'] = df[pid_col].map(infer_cohort)

    y = pd.to_numeric(df[y_col], errors='coerce')
    # si no es {0,1}, reescalar al valor máximo como 1
    uniq = sorted(pd.Series(y).dropna().unique().tolist())
    if len(uniq)==2 and set(uniq) != {0,1}:
        y = (y == max(uniq)).astype(int)

    out = pd.DataFrame({
        'patient_id': df[pid_col],
        'y_true': y.astype(int),
        'cohort': df['cohort']
    }).drop_duplicates('patient_id')

    assert out['y_true'].isin([0,1]).all(), f"Valores de y_true no binarios en {path_csv}: {out['y_true'].unique()}"
    return out



def load_features(file_list):
    acc = None
    for f in file_list:
        if not Path(f).exists():
            continue
        df = pd.read_csv(f)
        # Normaliza nombre id
        idcol = 'patient_id' if 'patient_id' in df.columns else None
        if idcol is None:
            # intenta inferir
            for c in df.columns:
                if re.match(r'patient[_]?id', c, re.I):
                    idcol = c; break
        if idcol is None:
            continue
        df = df.rename(columns={idcol:'patient_id'})
        # Limita a columnas numéricas + id
        numcols = ['patient_id'] + [c for c in df.columns if c!='patient_id' and pd.api.types.is_numeric_dtype(df[c])]
        df = df[numcols].drop_duplicates('patient_id')
        acc = df if acc is None else acc.merge(df, on='patient_id', how='outer')
    if acc is None:
        raise FileNotFoundError("No se cargaron features. Revisa rutas de p11/p14.")
    return acc

# --- Reemplazar build_Xy por esta versión (evita colisiones y_true / cohort) ---
def build_Xy(feat_df, lab_df):
    # elimina de features cualquier columna que pueda colisionar
    drop_like = {'y_true','y','target','label','truth','gt','cohort'}
    drop_cols = [c for c in feat_df.columns if c.lower() in drop_like]
    feat_df_clean = feat_df.drop(columns=drop_cols, errors='ignore').copy()

    # merge LEFT: labels mandan
    df = lab_df.merge(feat_df_clean, on='patient_id', how='left', validate='one_to_one')

    # y y cohorts siempre vienen del lab_df original
    y = df['y_true'].astype(int).values
    cohorts = df['cohort'].values

    # selecciona solo columnas numéricas de features
    blacklist = {'patient_id','y_true','cohort'}
    Xcols = [c for c in df.columns if c not in blacklist and pd.api.types.is_numeric_dtype(df[c])]
    X = df[Xcols].values

    return df[['patient_id','cohort','y_true']], X, y, Xcols


def sigmoid_platt_fit(p_raw, y):
    """Platt 'ligero': LR sobre prob como única feature."""
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression(solver='lbfgs', C=1.0, random_state=RANDOM_STATE)
    lr.fit(p_raw.reshape(-1,1), y)
    return lr

def sigmoid_platt_pred(model, p_raw):
    return model.predict_proba(p_raw.reshape(-1,1))[:,1]

def metrics(y_true, y_prob):
    return dict(
        AUC = roc_auc_score(y_true, y_prob) if len(np.unique(y_true))>1 else float('nan'),
        PRAUC = average_precision_score(y_true, y_prob) if len(np.unique(y_true))>1 else float('nan'),
        Brier = brier_score_loss(y_true, y_prob)
    )

# ---------- Carga ----------
labels_val  = load_labels(VAL_LABELS)
labels_test = load_labels(TEST_LABELS)

feat_val  = load_features(CANDIDATE_FEATURE_FILES_VAL)
feat_test = load_features(CANDIDATE_FEATURE_FILES_TEST)

meta_val,  X_val_raw,  y_val,  Xcols = build_Xy(feat_val, labels_val)
meta_test, X_test_raw, y_test, _    = build_Xy(feat_test, labels_test)

print(f"VAL: X={X_val_raw.shape}, TEST: X={X_test_raw.shape}, #features={len(Xcols)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
VAL: X=(69, 56), TEST: X=(70, 56), #features=56


In [4]:
# ---------- Modelo: LR elastic-net (saga) + scaler + imputer ----------
pipe = Pipeline([
    ('impute',  SimpleImputer(strategy='median')),
    ('scale',   StandardScaler(with_mean=True, with_std=True)),
    ('clf',     LogisticRegression(
        penalty='elasticnet', solver='saga', max_iter=5000, random_state=RANDOM_STATE
    ))
])

param_grid = {
    'clf__C': [0.03, 0.1, 0.3, 1.0, 3.0],
    'clf__l1_ratio': [0.1, 0.3, 0.5, 0.7]
}

rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=RANDOM_STATE)

def cv_score_auc(pipe, X, y, params):
    aucs = []
    for tr, va in rkf.split(X, y):
        m = Pipeline(pipe.steps)
        m.set_params(**params)
        m.fit(X[tr], y[tr])
        p = m.predict_proba(X[va])[:,1]
        aucs.append(roc_auc_score(y[va], p))
    return float(np.mean(aucs)), float(np.std(aucs))

# Grid search manual (robusto y transparente)
grid = list(ParameterGrid(param_grid))
scores = []
for params in grid:
    mu, sd = cv_score_auc(pipe, X_val_raw, y_val, params)
    scores.append({**params, 'cv_auc_mean':mu, 'cv_auc_std':sd})
cv_df = pd.DataFrame(scores).sort_values(['cv_auc_mean','cv_auc_std'], ascending=[False, True])
best = cv_df.iloc[0].to_dict()
print("Mejores hiperparámetros:", best)

# Entrenamiento final en VAL completo con los mejores hiperparámetros
best_pipe = Pipeline(pipe.steps)
best_pipe.set_params(**{k:v for k,v in best.items() if k.startswith('clf__')})
best_pipe.fit(X_val_raw, y_val)

# Probabilidades sin calibrar
p_val_raw  = best_pipe.predict_proba(X_val_raw)[:,1]
p_test_raw = best_pipe.predict_proba(X_test_raw)[:,1]

# Calibración Platt sobre VAL y aplicación a TEST
platt = sigmoid_platt_fit(p_val_raw, y_val)
p_val  = sigmoid_platt_pred(platt, p_val_raw)
p_test = sigmoid_platt_pred(platt, p_test_raw)

# Métricas globales
m_val_all  = metrics(y_val, p_val)
m_test_all = metrics(y_test, p_test)

# Métricas por cohorte
def by_cohort(meta, y, p):
    out=[]
    for coh in ['OAS1','OAS2']:
        idx = (meta['cohort'].values==coh)
        if idx.sum()==0:
            continue
        out.append({'Cohort':coh, **metrics(y[idx], p[idx])})
    return out

coh_val  = by_cohort(meta_val,  y_val,  p_val)
coh_test = by_cohort(meta_test, y_test, p_test)

# Guardado artefactos
pd.DataFrame({
    'patient_id': meta_val['patient_id'],
    'cohort': meta_val['cohort'],
    'y_true': meta_val['y_true'],
    'y_prob': p_val
}).to_csv(OUT/'p24_val_preds.csv', index=False)

pd.DataFrame({
    'patient_id': meta_test['patient_id'],
    'cohort': meta_test['cohort'],
    'y_true': meta_test['y_true'],
    'y_prob': p_test
}).to_csv(OUT/'p24_test_preds.csv', index=False)

# Coefs con nombres de features
clf = best_pipe.named_steps['clf']
scaler = best_pipe.named_steps['scale']
imputer = best_pipe.named_steps['impute']
coef = clf.coef_.ravel()
coef_df = pd.DataFrame({'feature': Xcols, 'coef': coef})
coef_df.to_csv(OUT/'p24_coefficients.csv', index=False)

with open(OUT/'p24_model.pkl','wb') as f:
    pickle.dump(best_pipe, f)
with open(OUT/'p24_platt.pkl','wb') as f:
    pickle.dump(platt, f)

summary = {
    'model': 'LogisticRegression(elastic-net, saga)',
    'best_params': {k:v for k,v in best.items() if k.startswith('clf__')},
    'cv_auc_mean': best['cv_auc_mean'], 'cv_auc_std': best['cv_auc_std'],
    'val_global': m_val_all, 'test_global': m_test_all,
    'val_by_cohort': coh_val, 'test_by_cohort': coh_test,
    'n_features': len(Xcols), 'random_state': RANDOM_STATE
}
with open(OUT/'p24_summary.json','w') as f:
    json.dump(summary, f, indent=2)

print("✅ P24 artefactos guardados en:", OUT)
print("CV(top):", best['cv_auc_mean'], "±", best['cv_auc_std'])
print("VAL ALL:", m_val_all, "\nTEST ALL:", m_test_all)
print("VAL by cohort:", coh_val, "\nTEST by cohort:", coh_test)

Mejores hiperparámetros: {'clf__C': 0.1, 'clf__l1_ratio': 0.7, 'cv_auc_mean': 0.8796428571428572, 'cv_auc_std': 0.09015979022609164}
✅ P24 artefactos guardados en: /content/drive/MyDrive/CognitivaAI/p24_meta_simple
CV(top): 0.8796428571428572 ± 0.09015979022609164
VAL ALL: {'AUC': np.float64(0.9083191850594228), 'PRAUC': np.float64(0.9250122566193104), 'Brier': np.float64(0.1760208111803272)} 
TEST ALL: {'AUC': np.float64(0.7269736842105263), 'PRAUC': np.float64(0.7167798547072909), 'Brier': np.float64(0.2198784142685081)}
VAL by cohort: [{'Cohort': 'OAS1', 'AUC': np.float64(0.9092592592592592), 'PRAUC': np.float64(0.9207944528875383), 'Brier': np.float64(0.17067739931475404)}, {'Cohort': 'OAS2', 'AUC': np.float64(0.9008264462809917), 'PRAUC': np.float64(0.9285204991087346), 'Brier': np.float64(0.1874362819840517)}] 
TEST by cohort: [{'Cohort': 'OAS1', 'AUC': np.float64(0.7537037037037037), 'PRAUC': np.float64(0.7358583509975016), 'Brier': np.float64(0.21077937941445724)}, {'Cohort': '

In [5]:
import json, math
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

BASE_DIR = Path("/content/drive/MyDrive/CognitivaAI")
OUT = BASE_DIR / "p24_meta_simple"

val = pd.read_csv(OUT/"p24_val_preds.csv")   # contiene y_prob calibrado (Platt)
test = pd.read_csv(OUT/"p24_test_preds.csv")

C_FN, C_FP = 5.0, 1.0

def metrics(y_true, y_prob, thr=0.5):
    y_pred = (y_prob>=thr).astype(int)
    tp = int(((y_pred==1)&(y_true==1)).sum())
    fp = int(((y_pred==1)&(y_true==0)).sum())
    tn = int(((y_pred==0)&(y_true==0)).sum())
    fn = int(((y_pred==0)&(y_true==1)).sum())
    auc = roc_auc_score(y_true, y_prob) if len(np.unique(y_true))>1 else float('nan')
    pr  = average_precision_score(y_true, y_prob) if len(np.unique(y_true))>1 else float('nan')
    brier = brier_score_loss(y_true, y_prob)
    prec = tp/(tp+fp) if (tp+fp)>0 else 0.0
    rec  = tp/(tp+fn) if (tp+fn)>0 else 0.0
    acc  = (tp+tn)/(tp+tn+fp+fn)
    cost = C_FN*fn + C_FP*fp
    return dict(AUC=auc, PRAUC=pr, Brier=brier, Acc=acc, Precision=prec, Recall=rec, TP=tp, FP=fp, TN=tn, FN=fn, Cost=cost)

def best_threshold_cost(y_true, y_prob):
    grid = np.linspace(0,1,1001)
    best = (0.5, math.inf)
    for t in grid:
        y_pred = (y_prob>=t).astype(int)
        fn = int(((y_pred==0)&(y_true==1)).sum())
        fp = int(((y_pred==1)&(y_true==0)).sum())
        cost = C_FN*fn + C_FP*fp
        if cost < best[1]: best = (float(t), float(cost))
    return best

thr_json = {}
rows = []

for cohort in ["OAS1","OAS2"]:
    v = val[val['cohort']==cohort]
    t = test[test['cohort']==cohort]
    if len(v)==0 or len(t)==0:
        continue
    thr, cost_val = best_threshold_cost(v['y_true'].values, v['y_prob'].values)
    m_val  = metrics(v['y_true'].values, v['y_prob'].values, thr)
    m_test = metrics(t['y_true'].values, t['y_prob'].values, thr)
    m_val.update({"Cohort":cohort, "Thr":thr})
    m_test.update({"Cohort":cohort, "Thr":thr})
    rows.append({"split":"VAL", **m_val})
    rows.append({"split":"TEST", **m_test})
    thr_json[cohort] = {"thr":thr, "cost_val":cost_val, "C_FN":C_FN, "C_FP":C_FP}

rep = pd.DataFrame(rows)
rep.to_csv(OUT/"p24_test_report.csv", index=False)
with open(OUT/"p24_thresholds.json","w") as f:
    json.dump(thr_json, f, indent=2)

print("Guardado:")
print("-", OUT/"p24_test_report.csv")
print("-", OUT/"p24_thresholds.json")
display(rep)


Guardado:
- /content/drive/MyDrive/CognitivaAI/p24_meta_simple/p24_test_report.csv
- /content/drive/MyDrive/CognitivaAI/p24_meta_simple/p24_thresholds.json


,split,AUC,PRAUC,Brier,Acc,Precision,Recall,TP,FP,TN,FN,Cost,Cohort,Thr
0,VAL,0.909259,0.920794,0.170677,0.851064,0.760000,0.950000,19,6,21,1,11.0,OAS1,0.435
1,TEST,0.753704,0.735858,0.210779,0.680851,0.608696,0.700000,14,9,18,6,39.0,OAS1,0.435
2,VAL,0.900826,0.928520,0.187436,0.727273,0.647059,1.000000,11,6,5,0,6.0,OAS2,0.332
3,TEST,0.750000,0.804859,0.238472,0.652174,0.611111,0.916667,11,7,4,1,12.0,OAS2,0.332


In [6]:
import json, pandas as pd
from pathlib import Path
from textwrap import dedent

BASE_DIR = Path("/content/drive/MyDrive/CognitivaAI")
OUT = BASE_DIR / "p24_meta_simple"

sumy = json.load(open(OUT/"p24_summary.json"))
test_coh = pd.DataFrame(sumy['test_by_cohort'])  # ya lo imprimiste antes

# Si ejecutaste la celda de umbrales, leemos el reporte
try:
    report = pd.read_csv(OUT/"p24_test_report.csv")
    rep_test = report[report['split']=="TEST"].copy()
except FileNotFoundError:
    report = None
    rep_test = None

def fmt(x, d=3):
    try: return f"{float(x):.{d}f}"
    except: return str(x)

# README.md
md_readme = dedent(f"""
### P24 — Meta simple y robusto (LR elastic-net + KFold repetido)

**Mejores hiperparámetros (CV 5×5):** {sumy['best_params']}
**CV AUC:** {fmt(sumy['cv_auc_mean'])} ± {fmt(sumy['cv_auc_std'])}

**Resultados (TEST, probabilidades calibradas con Platt):**
- **Global:** AUC={fmt(sumy['test_global']['AUC'])} | PR-AUC={fmt(sumy['test_global']['PRAUC'])} | Brier={fmt(sumy['test_global']['Brier'])}
- **OAS1:** AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','AUC'].values[0])} | PR-AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','PRAUC'].values[0])} | Brier={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','Brier'].values[0])}
- **OAS2:** AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','AUC'].values[0])} | PR-AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','PRAUC'].values[0])} | Brier={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','Brier'].values[0])}

{"" if rep_test is None else f"**Umbrales coste-óptimos (FN=5, FP=1):** " + ", ".join([f"{r['Cohort']} thr={fmt(r['Thr'],3)} → Coste={fmt(r['Cost'],1)} | R={fmt(r['Recall'],2)} | P={fmt(r['Precision'],2)} | Acc={fmt(r['Acc'],2)}" for _,r in rep_test.iterrows()])}

_Artefactos_: `p24_val_preds.csv`, `p24_test_preds.csv`, `p24_coefficients.csv`, `p24_model.pkl`, `p24_platt.pkl`, `p24_summary.json`{"" if rep_test is None else ", `p24_thresholds.json`, `p24_test_report.csv`"}.
""").strip()

# InformeTecnico.md
md_informe = dedent(f"""
## P24 — Meta interpretable (LR elastic-net)

**Diseño:** fusión de features paciente (catálogo p11 + OAS2 p14), LR (elastic-net, saga), **RepeatedStratifiedKFold** 5×5, calibración Platt, evaluación por cohorte.

**CV (5×5):** AUC={fmt(sumy['cv_auc_mean'])} ± {fmt(sumy['cv_auc_std'])} | Parámetros: {sumy['best_params']}

**Resultados (TEST):**
- Global: AUC={fmt(sumy['test_global']['AUC'])} | PR-AUC={fmt(sumy['test_global']['PRAUC'])} | Brier={fmt(sumy['test_global']['Brier'])}
- OAS1: AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','AUC'].values[0])} | PR-AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','PRAUC'].values[0])} | Brier={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','Brier'].values[0])}
- OAS2: AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','AUC'].values[0])} | PR-AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','PRAUC'].values[0])} | Brier={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','Brier'].values[0])}

{"" if rep_test is None else "**Decisión (coste clínico, FN=5, FP=1):**  " + " · ".join([f"{r['Cohort']} thr={fmt(r['Thr'],3)} → Coste={fmt(r['Cost'],1)} (R={fmt(r['Recall'],2)}, P={fmt(r['Precision'],2)}, Acc={fmt(r['Acc'],2)})" for _,r in rep_test.iterrows()])}

**Interpretación:** frente a P23, P24 recupera **discriminación en OAS2** (AUC≈0.75) sosteniendo OAS1. El meta simple + calibración ofrece probabilidades fiables y coeficientes interpretables.
""").strip()

# Bitácora
from datetime import datetime
today = datetime.now().strftime("%Y-%m-%d")
md_bitacora = dedent(f"""
### {today} — P24 ejecutado (LR elastic-net + KFold repetido + Platt)

- Features paciente fusionadas (p11+p14).
- CV(5×5): AUC={fmt(sumy['cv_auc_mean'])}±{fmt(sumy['cv_auc_std'])}; mejores params: {sumy['best_params']}.
- TEST Global: AUC={fmt(sumy['test_global']['AUC'])}, PR-AUC={fmt(sumy['test_global']['PRAUC'])}, Brier={fmt(sumy['test_global']['Brier'])}.
- TEST OAS1: AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','AUC'].values[0])}, PR-AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','PRAUC'].values[0])}, Brier={fmt(test_coh.loc[test_coh['Cohort']=='OAS1','Brier'].values[0])}.
- TEST OAS2: AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','AUC'].values[0])}, PR-AUC={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','PRAUC'].values[0])}, Brier={fmt(test_coh.loc[test_coh['Cohort']=='OAS2','Brier'].values[0])}.
{"" if rep_test is None else "- Umbrales coste per-cohorte: " + " | ".join([f"{r['Cohort']} thr={fmt(r['Thr'],3)} → Coste={fmt(r['Cost'],1)} (R={fmt(r['Recall'],2)}, P={fmt(r['Precision'],2)}, Acc={fmt(r['Acc'],2)})" for _,r in rep_test.iterrows()])}

_Artefactos_: `p24_meta_simple/` (preds, coeficientes, modelo, calibrador, summary{"" if rep_test is None else ", thresholds, report"}).
""").strip()

print("\n--- README.md (bloque) ---\n")
print(md_readme)
print("\n--- InformeTecnico.md (bloque) ---\n")
print(md_informe)
print("\n--- CuadernoBitacora.md (bloque) ---\n")
print(md_bitacora)



--- README.md (bloque) ---

### P24 — Meta simple y robusto (LR elastic-net + KFold repetido)

**Mejores hiperparámetros (CV 5×5):** {'clf__C': 0.1, 'clf__l1_ratio': 0.7}  
**CV AUC:** 0.880 ± 0.090

**Resultados (TEST, probabilidades calibradas con Platt):**
- **Global:** AUC=0.727 | PR-AUC=0.717 | Brier=0.220
- **OAS1:** AUC=0.754 | PR-AUC=0.736 | Brier=0.211
- **OAS2:** AUC=0.750 | PR-AUC=0.805 | Brier=0.238

**Umbrales coste-óptimos (FN=5, FP=1):** OAS1 thr=0.435 → Coste=39.0 | R=0.70 | P=0.61 | Acc=0.68, OAS2 thr=0.332 → Coste=12.0 | R=0.92 | P=0.61 | Acc=0.65

_Artefactos_: `p24_val_preds.csv`, `p24_test_preds.csv`, `p24_coefficients.csv`, `p24_model.pkl`, `p24_platt.pkl`, `p24_summary.json`, `p24_thresholds.json`, `p24_test_report.csv`.

--- InformeTecnico.md (bloque) ---

## P24 — Meta interpretable (LR elastic-net)

**Diseño:** fusión de features paciente (catálogo p11 + OAS2 p14), LR (elastic-net, saga), **RepeatedStratifiedKFold** 5×5, calibración Platt, evaluación por coho